In [12]:
from importlib import reload
import sys
from imp import reload
import warnings 
warnings.filterwarnings('ignore')

In [13]:
import pandas as pd

df = pd.read_csv('imdb_master.csv', encoding="latin-1")
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [14]:
df = df.drop(['Unnamed: 0','type','file'],axis=1)
df.columns = ["review","sentiment"]
df.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [15]:
df = df[df.sentiment != 'unsup']
df['sentiment'] = df['sentiment'].map({'pos':1, 'neg': 0})
df.head()

,review,sentiment
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0


In [17]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] = df.review.apply(lambda x: clean_text(x))
df.head()

,review,sentiment,Processed_Reviews
0,Once again Mr. Costner has dragged out a movie...,0,mr costner ha drag movie far longer necessary ...
1,This is an example of why the majority of acti...,0,example majority action film generic bore real...
2,"First of all I hate those moronic rappers, who...",0,first hate moronic rapper couldnt act gun pres...
3,Not even the Beatles could write songs everyon...,0,even beatles could write song everyone like al...
4,Brass pictures (movies is not a fitting word f...,0,brass picture movie fit word really somewhat b...


In [19]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

127.99056

In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(df['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Using TensorFlow backend.


Train on 40000 samples, validate on 10000 samples
Epoch 1/3
40000/40000 [==============================] - 114s 3ms/step - loss: 0.3783 - acc: 0.8241 - val_loss: 0.4107 - val_acc: 0.8242
Epoch 2/3
40000/40000 [==============================] - 110s 3ms/step - loss: 0.2340 - acc: 0.9076 - val_loss: 0.4665 - val_acc: 0.7983
Epoch 3/3
40000/40000 [==============================] - 112s 3ms/step - loss: 0.1909 - acc: 0.9274 - val_loss: 0.4676 - val_acc: 0.8107


In [ ]:
test_input = input()


In [22]:
df3 = pd.DataFrame({'review': [test_input],
                   'sentiment': ['']})
df3.head()

,review,sentiment
0,This movie is a disaster within a disaster fil...,


In [25]:
y_test = df3["sentiment"]
y_test
list_sentences_test = df3["review"]
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
prediction

array([[0.10687274]], dtype=float32)